In [100]:
from collections import OrderedDict
import copy


In [101]:
def printTablero(tablero: list[list[str]]):
    print(tablero_to_str(tablero))

def tablero_to_str(tablero: list[list[str]]) -> str:
    """
    Convertimos el tablero en una representación visual de string.
    """
    string = "\n".join([" ".join(i) for i in tablero])
    return string


In [102]:
def to_tup(tablero: list[list[int]]) -> tuple[tuple[int]]:
    """
    Conversión de una matriz en forma de listas a forma de tuplas.
    """
    tablero = map(lambda x: tuple(x), tablero)
    return tuple(tablero)
    
def to_list(tablero: tuple[tuple[int]]) -> list[list[int]]:
    """
    Conversión de una matriz en forma de tuplas a forma de listas.
    """
    tablero = list(tablero)
    tablero = list(map(lambda x: list(x), tablero))
    return tablero

In [103]:
def get_empty_coord(tablero: list[list[int]]) -> tuple[int, int]:
    """
    Dado un tablero, se obtiene la coordenada de la casilla vacía, demarcada como '-'.
    """
    for i in range(len(tablero)):
        for j in range(len(tablero[i])):
            if tablero[i][j] == "-":
                return i, j
            
def get_neighbor_coords(estadoTableroActual: list[list[int]]) -> list[tuple[int, int]]:
    """
    Encuentra las coordenadas de los vecinos válidos de la casilla actual.
    """
    current_empty_coord = get_empty_coord(estadoTableroActual)
    neighbors: list[tuple[int, int]] = []
    directions: list[tuple[int, int]] = [(-1, 0), (1, 0), (0, -1), (0, 1)] # Arriba, Abajo,  Izquierda, Derecha
    for direction in directions:
        neighbor = current_empty_coord[0] + direction[0], current_empty_coord[1] + direction[1]
        if 0 <= neighbor[0] < len(estadoTableroActual) and 0 <= neighbor[1] < len(estadoTableroActual[0]):
            neighbors.append(neighbor)
    # print("neighbors:", neighbors)
    return neighbors

def switchTablero(swap_coords: tuple[int, int], tablero: list[list[str]]) -> list[list[str]]:
    """
    Dadas las coordinadas a intercambiar 'swap_coords', y el estado original de un 'tablero',
    se regresa otro tablero, donde el espacio vacío '-' del tablero es intercambiado de lugar
    con la casilla dada por 'swap_coords'.
    """
    # Hacemos la copia del tablero original, y es la cual vamos a regresar de la función.
    newTablero = to_list(copy.deepcopy(tablero))
    oldX, oldY = get_empty_coord(newTablero) # obtenemos las coordenadas del espacio '_'. 

    # [(-1, 0), (1, 0), (0, -1), (0, 1)] # Arriba, Abajo,  Izquierda, Derecha.
    newX, newY = swap_coords[0], swap_coords[1]

    # Se hace el cambio de valores con una variable temporal.
    temp = newTablero[newX][newY]
    newTablero[newX][newY] = newTablero[oldX][oldY]
    newTablero[oldX][oldY] = temp
    return newTablero


In [104]:
 
def fun_heurística(estado_actual: list[list[str]], estado_objetivo: list[list[str]]) -> int:
    """
    Función heruística utilizada en la búsqueda. Por cada casilla del estado 
    actual que no concuerde con la casilla del estado objetivo, se añade coste.
    """
    casillas_desacomodadas = 0
    for s, g in zip(estado_actual, estado_objetivo):
        for h, k in zip(s, g):
            if h != k:
                casillas_desacomodadas += 1
    return casillas_desacomodadas


In [105]:
def make_ordered_list_of_states(start_state: tuple[tuple[int]],
                                end_state: tuple[tuple[int]],
                                came_from: dict[tuple[tuple[str]], tuple[tuple[str]]]) -> list[tuple[tuple[int]]]:
    """
    Dada la lista de procedencias `came_from` y un estado inicial y final, se regresa 
    el camino del estado inicial al final de forma: `[start_state, ..., end_state]`.
    """
    state_list: list[tuple[tuple[int]]] = [end_state]
    last = end_state
    while last != start_state:
        last = came_from[last]
        state_list.append(last)
    return list(reversed(state_list))

def print_camino(state_history: list[tuple[tuple[str]]]):
    """
    Se imprime la visualización del camino
    """
    arrow_text = """
      :::
      ;;;
    ..;;;..
     ':::'
       '
    """
    string = (arrow_text+ "\n").join(map(tablero_to_str, state_history))
    print(f"\n{'='*10}")
    print("RESULTADO DE LA BÚSQUEDA:")
    print(string)
    print("="*10)


In [106]:
def a_star(tablero_inicial: list[list[int]], tablero_objetivo: list[list[int]], imprimir=False) -> list[tuple[tuple[str]]]:
    """
    Implementa el algoritmo A* para encontrar el camino a un tablero, dados un tablero inicial
    y un tablero objetivo. Devuelve la lista de los estados ordenados, del tablero inicial al 
    tablero objetivo. Si está vacía, no se encontró un camino.
    """
    tablero_inicial = to_tup(tablero_inicial)
    tablero_objetivo = to_tup(tablero_objetivo)

    # Las llaves son los estados del tablero, y sus valores son su costo g (movimientos previos + 1)
    #  y h (costo heurístico: número de casillas incorrectas, menor es mejor)
    # aquellos en `open_list` son los nodos a explorar
    open_list: OrderedDict[tuple[tuple[int]], tuple[int, int]] = OrderedDict({tablero_inicial: (0, fun_heurística(tablero_inicial, tablero_objetivo))})

    # Diccionario de los costos reales (número de movientos, desde el tablero inicial) para llegar a cierto estado.
    g_scores: dict[tuple[tuple[int]],int] = {tablero_inicial: 0}
    
    # Diccionario que almacena que estado es el anterior: {estado1: estado2}, el estado1 viene del estado2
    came_from: dict[tuple[tuple[int]],tuple[tuple[int]]] = {}

    while open_list:
        # Se ordena para que los elementos con menor coste estén al final del diccionario: {estado1: (1,1), estado2: (1,3)} -> {estado2: (1,3), estado1: (1,1)} 
        open_list = OrderedDict(sorted(open_list.items(), key=lambda estado: estado[1][1], reverse=True))

        current_tablero, current_tablero_cost = open_list.popitem()
        if imprimir is True:
            print("----------------------")
            print("Estado actual:")
            printTablero(current_tablero)
        
        if current_tablero == tablero_objetivo:
            # Se reconstruye el caminio, terminando la iteración y regresamos el resultado final de la búsqueda
            return make_ordered_list_of_states(tablero_inicial, tablero_objetivo, came_from)
        
        for neighbor_state in map(lambda neigh: switchTablero(neigh, current_tablero), get_neighbor_coords(current_tablero)):
            neighbor_state = to_tup(neighbor_state)
            # Costo (tentativo) de moverse al vecino
            tentative_g_score_of_current_neighbor_state = g_scores[current_tablero] + 1 
            if (neighbor_state not in g_scores) or (tentative_g_score_of_current_neighbor_state < g_scores[neighbor_state]):
                # si (no se ha llegado a este estado antes) o (el costo de este camino a esete estado es menor a costos encontrados antes), entonces:

                # añadimos a la lista de procedencia a los estados vecinos, indicando que vendrían del current_tablero
                came_from[neighbor_state] = current_tablero

                # añadimos el (hasta ahora) mejor costo encontrado para llegar a neighbor_state
                g_scores[neighbor_state] = tentative_g_score_of_current_neighbor_state

                # añadimos el vecino con su costo heurístico a la lista de estados por explorar
                f_score = tentative_g_score_of_current_neighbor_state + fun_heurística(neighbor_state, tablero_objetivo)
                open_list[neighbor_state] = (tentative_g_score_of_current_neighbor_state, f_score)

                if imprimir is True:
                    print("Estado vecino")
                    printTablero(neighbor_state)
                    print("Costo", f_score)
    return []


# =================================
# EJECUCIÓN
# =================================

In [107]:
tablero_inicial: list[list[str]] = [
    ["-", "1", "3"],
    ["4", "5", "8"],
    ["6", "7", "2"],
]

tablero_objetivo: list[list[str]] = [
    ["1", "2", "3"],
    ["4", "5", "6"],
    ["7", "8", "-"],
]


In [108]:
def main():
    print("----------------------")
    print("Condiciones Iniciales:")
    print("Estado inicial:")
    printTablero(tablero_inicial)
    print("\nEstado final:")
    printTablero(tablero_objetivo)

    state_history = a_star(tablero_inicial, tablero_objetivo, imprimir=False)
    if len(state_history) == 0:
        print("No se encontró ningún camino al objetivo")
    else:
        # Se imprime la visualización del camino
        print_camino(state_history)
        
main()

----------------------
Condiciones Iniciales:
Estado inicial:
- 1 3
4 5 8
6 7 2

Estado final:
1 2 3
4 5 6
7 8 -

RESULTADO DE LA BÚSQUEDA:
- 1 3
4 5 8
6 7 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
- 5 8
6 7 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
5 - 8
6 7 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
5 7 8
6 - 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
5 7 8
- 6 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
- 7 8
5 6 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 - 8
5 6 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 6 8
5 - 2
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 6 8
5 2 -
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 6 -
5 2 8
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 - 6
5 2 8
      :::
      ;;;
    ..;;;..
     ':::'
       '
    
4 1 3
7 2 6
5 - 8
      :::
      ;